In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from torchvision import transforms
from scipy.ndimage import zoom
from tqdm import tqdm
import os
import re

from shapely.geometry import box
import geopandas as gpd
import rasterio
from rasterio.transform import from_origin
from shapely.ops import unary_union
from fiona.crs import from_epsg
import os
import numpy as np
from geopy.distance import geodesic
import pandas as pd
import argparse

from sentinelhub import SentinelHubRequest, SentinelHubDownloadClient, BBox, CRS, DataCollection, MimeType, SHConfig
from dotenv import load_dotenv
import argparse
import os
import json
from datetime import datetime, timedelta
from tqdm import tqdm
import numpy as np
import geojson

In [2]:
class UNet(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(UNet, self).__init__()

        self.encoder = nn.Sequential(
            nn.Conv2d(in_channels, 64, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(64, 64, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        
        self.decoder = nn.Sequential(
            nn.Conv2d(64, 64, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(64, out_channels, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Upsample(scale_factor=2, mode='bilinear', align_corners=True)
        )

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x



In [3]:
#load model
model = UNet(13, 1)
model.load_state_dict(torch.load("unet_model.pth"))

<All keys matched successfully>

In [4]:
#generate square boxes from geojson

def meters_to_degrees(meters, latitude):
    # Calculate the distance per degree at the specified latitude
    meters_per_degree = geodesic((latitude, 0), (latitude + 1, 0)).meters

    # Convert meters to degrees
    degrees = meters / meters_per_degree

    return degrees

def get_bounds(geometry):
    coords = np.array(list(geojson.utils.coords(geometry)))
    return coords[:,0].min(), coords[:,0].max(), coords[:,1].min(), coords[:,1].max()

def generate_bounding_boxes(geojson_path, box_size):
    """
    Generate fixed-sized square bounding boxes inside the polygons of a GeoJSON file.

    Parameters:
    - geojson_path: Path to the GeoJSON file
    - box_size: Size of the square bounding boxes

    Returns:
    - List of Shapely Polygon objects representing the bounding boxes
    """
    # Open the GeoJSON file and get its polygons
    bounding_boxes = []
    

    gdf = gpd.read_file(geojson_path)
    
    for i, polygon in enumerate(gdf.geometry):
        avg_lat = polygon.centroid.y
        box_size_deg = meters_to_degrees(box_size, avg_lat)

        # Calculate the bounding box
        bounds = polygon.bounds
        num_boxes_x = int((bounds[2] - bounds[0]) // box_size_deg)
        num_boxes_y = int((bounds[3] - bounds[1]) // box_size_deg)
        
        for j in range(num_boxes_x):
            for k in range(num_boxes_y):
                min_x = bounds[0] + j * box_size_deg
                min_y = bounds[1] + k * box_size_deg
                max_x = min_x + box_size_deg
                max_y = min_y + box_size_deg

                box_geometry = box(min_x, min_y, max_x, max_y)

                # Check if the box is completely within the polygon
                if polygon.contains(box_geometry):
                    
                    bounding_boxes.append(box_geometry)

    return bounding_boxes

In [5]:
bounding_boxes = generate_bounding_boxes("../data/bbox/bbox_1.json", 1000)

In [6]:
bounding_boxes

[<POLYGON ((139.813 -18.539, 139.813 -18.53, 139.804 -18.53, 139.804 -18.539,...>,
 <POLYGON ((139.813 -18.53, 139.813 -18.521, 139.804 -18.521, 139.804 -18.53,...>,
 <POLYGON ((139.813 -18.521, 139.813 -18.512, 139.804 -18.512, 139.804 -18.52...>,
 <POLYGON ((139.822 -18.539, 139.822 -18.53, 139.813 -18.53, 139.813 -18.539,...>,
 <POLYGON ((139.822 -18.53, 139.822 -18.521, 139.813 -18.521, 139.813 -18.53,...>,
 <POLYGON ((139.822 -18.521, 139.822 -18.512, 139.813 -18.512, 139.813 -18.52...>,
 <POLYGON ((139.831 -18.539, 139.831 -18.53, 139.822 -18.53, 139.822 -18.539,...>,
 <POLYGON ((139.831 -18.53, 139.831 -18.521, 139.822 -18.521, 139.822 -18.53,...>,
 <POLYGON ((139.831 -18.521, 139.831 -18.512, 139.822 -18.512, 139.822 -18.52...>]

In [7]:
target_crs = from_epsg(4326)    
gdf = gpd.GeoDataFrame(geometry=bounding_boxes, crs=target_crs)

In [8]:
gdf.to_file('bounding_boxes_for_preds.geojson', driver='GeoJSON') 

In [9]:
size = (512, 512)
time_interval = "2023-08-30", "2023-08-31"
load_dotenv()

config = SHConfig()
config.sh_client_id = os.getenv("CLIENT_ID")
config.sh_client_secret = os.getenv("CLIENT_SECRET")

def getL2A():
    evalscript_true_color ="""//VERSION=3

    function setup() {
        return {
            input: [{
                bands: ["B02", "B03", "B04", "B05", "B06", "B07", "B8A", "B09", "B11", "B12", "CLM", "dataMask"],
            }],
            output: {
                bands: 11
            }
        };
    }

    function evaluatePixel(sample) {
        if (sample.dataMask != 1) {
            return [99, 99, 99]
        }
        return [
            2.5 * sample.B02,
            2.5 * sample.B03,
            2.5 * sample.B04,
            2.5 * sample.B05,
            2.5 * sample.B06,
            2.5 * sample.B07,
            2.5 * sample.B8A,
            2.5 * sample.B09,
            2.5 * sample.B11,
            2.5 * sample.B12,
            sample.CLM,
        ];
    }"""


    request = SentinelHubRequest(
        data_folder = data_folder,
        evalscript=evalscript_true_color,
        input_data=[
            SentinelHubRequest.input_data(
                data_collection=DataCollection.SENTINEL2_L2A,
                time_interval=time_interval,
                mosaicking_order="leastCC",
    
            )
        ],
        responses=[SentinelHubRequest.output_response("default", MimeType.TIFF)],
        bbox=bbox,
        size=size,
        config=config,
    )

    image = request.get_data()[0]
    
    return image

def getDEM():
    evalscript_true_color = """
    //VERSION=3

    function setup() {
        return {
            input: [{
                bands: ["DEM"]
            }],
            output: {
                bands: 1,
                sampleType: "FLOAT32" 
            }
        };
    }

    function evaluatePixel(sample) {
        return [sample.DEM];
    }
    """

    request = SentinelHubRequest(
        data_folder = data_folder,
        evalscript=evalscript_true_color,
        input_data=[
            SentinelHubRequest.input_data(
                data_collection=DataCollection.DEM,
                time_interval=time_interval,
                mosaicking_order="leastCC"

            )
        ],
        responses=[SentinelHubRequest.output_response("default", MimeType.TIFF)],
        bbox=bbox,
        size=size,
        config=config,
    )

    dem = request.get_data()[0]
    return dem


In [10]:
config.sh_client_id, config.sh_client_secret

('714d02a0-e57f-46ca-8e64-2e79780d8e43', 'Tfe713aSRJyLs28rvMP1tYUOnX7ivDpR')

In [11]:
with open("bounding_boxes_for_preds.geojson") as file:
    data = json.load(file)

In [12]:
print("Downloading Sentinel 2 Bands...")
root = "../preds"
if not os.path.isdir(root):
    os.mkdir(root)
data_folder = "../preds/bboxes"
if not os.path.isdir(data_folder):
    os.mkdir(data_folder)
for i in tqdm(range(len(data['features']))):
    coords = data['features'][i]['geometry']['coordinates'][0]
    bl = (min(row[0] for row in coords), min(row[1] for row in coords))
    tr = (max(row[0] for row in coords), max(row[1] for row in coords))
    
    bbox = BBox((bl, tr), crs=CRS.WGS84)  
    
    image = getL2A()
    dem = getDEM()
    sub = (image[..., 3] - image[..., 2]).astype(float)
    add = (image[..., 3] + image[..., 2]).astype(float)
    ndvi = np.divide(sub, add, out = np.zeros_like(add) - 1, where = add != 0)
    ndvi = np.clip(ndvi, -1, 1)
    dem3 = dem[:, :, np.newaxis]
    ndvi3 = ndvi[:, :, np.newaxis]
    
    final_image = np.concatenate((image, dem3, ndvi3), axis = -1)
    np.save(data_folder + "/{iid}.npy".format(iid = i), final_image)

100%|█████████████████████████████████████████████████████████████████████████████████████| 9/9 [00:26<00:00,  2.91s/it]


In [13]:
preds_bboxes_path = '../preds/bboxes/'
preds_bboxes_npy_path = [os.path.join(preds_bboxes_path, file) for file in os.listdir(preds_bboxes_path) if file.endswith(".npy")]

In [16]:
preds_bboxes_npy_path = sorted(preds_bboxes_npy_path)
preds_bboxes_npy_path

['../preds/bboxes/0.npy',
 '../preds/bboxes/1.npy',
 '../preds/bboxes/2.npy',
 '../preds/bboxes/3.npy',
 '../preds/bboxes/4.npy',
 '../preds/bboxes/5.npy',
 '../preds/bboxes/6.npy',
 '../preds/bboxes/7.npy',
 '../preds/bboxes/8.npy']

In [17]:
def interpolate(array, target_size):
    zoom_factors = [target_size[i] / array.shape[i] for i in range(2)]
    zoom_factors.append(1)

    interpolated_array = zoom(array, zoom_factors, order=1, mode='nearest')

    return interpolated_array

In [18]:
#model eval
model.eval()

UNet(
  (encoder): Sequential(
    (0): Conv2d(13, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (decoder): Sequential(
    (0): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 1, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): Upsample(scale_factor=2.0, mode=bilinear)
  )
)

In [21]:
data_folder = "../preds/labels"
if not os.path.isdir(data_folder):
    os.mkdir(data_folder)

for i, path in enumerate(preds_bboxes_npy_path):    
    im = np.load(path)
    im = im.astype(np.float32)
    im = interpolate(im, (512, 512))
    im[..., 0 : 3] /= 255
    im = np.reshape(im, (1, 13, 512, 512))
    im = torch.from_numpy(im)
    with torch.no_grad():
        predicted_output = model(im)

    res = predicted_output.numpy()
    res = np.reshape(res, (512, 512, 1))
    np.save("../preds/labels/{iid}.npy".format(iid = i), res)